## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [195]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

bank_data = pd.read_csv('/datasets/data.csv')

print(bank_data.head(10))
print()

print('Общая информация:')
bank_data.info()
print()

print('Размер датасета:', bank_data.shape)
print()
print('Названия столбцов:', bank_data.columns)

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

In [196]:
# учитывая комментарии применим метод describe и sample для закрепления материала :)

bank_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [197]:
bank_data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2742,0,-1198.280914,43,высшее,0,в разводе,3,M,госслужащий,0,191350.461476,строительство жилой недвижимости
1240,0,-1438.334957,42,среднее,1,женат / замужем,0,M,сотрудник,1,135588.196177,покупка коммерческой недвижимости
11850,0,NaN,61,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,покупка жилья
11440,2,-2323.345626,39,среднее,1,женат / замужем,0,M,компаньон,0,215335.937646,операции со своей недвижимостью
7300,0,-1005.889027,56,Среднее,1,женат / замужем,0,M,компаньон,0,284776.525280,покупка жилья


### Вывод

Открыл файл, получил представление о данных, размер датасета. В файле 2 столбца в формате float64, 5 столбцов в формате int64 и 5 столобцов формата object

### Шаг 2. Предобработка данных

### Обработка пропусков

In [198]:
# выделим столбцы, необходимые для ответа на поставленые вопросы:
# children, family_status_id, debt, total_income, purpose
# выявляем пропуски в столбцах
print('Пропуски:')
print(bank_data.isna().sum())
# в столбце total_income, нужном нам для анализа, есть 2174 пропуска
# посчитаем медиану по столбцу total_income в зависимости от типа занятости и заменим пропуски на медиану
# столбец days_employed по пропускам сообветствует столбцу total_income,
# пропуски также заменяем медианой в зависимости от типа занятости, но этот столбец нам не сильно интересен, т. е.
# в нём нет данных, необходимых для ответов на поставленные вопросы

for income in bank_data['income_type'].unique():
    median = bank_data.loc[bank_data['income_type'] == income, 'total_income'].median()
    bank_data.loc[(bank_data['total_income'].isna()) & (bank_data['income_type'] == income), 'total_income'] = median
    
for income in bank_data['income_type'].unique():
    median = bank_data.loc[bank_data['income_type'] == income, 'days_employed'].median()
    bank_data.loc[(bank_data['days_employed'].isna()) & (bank_data['income_type'] == income), 'days_employed'] = median

# проанализуруем столбец family_status_id на уникальные значения
print('Уникальные значения столбца family_status_id:', bank_data['family_status_id'].unique())
# как можем видеть значения от 0 до 4, пропусков нет - значит нас всё устраивает

# проанализуруем столбец debt на уникальные значения
print('Уникальные значения столбца debt:', bank_data['debt'].unique())
# как можем видеть значения 0 и 1, пропусков нет - значит нас всё устраивает

Пропуски:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Уникальные значения столбца family_status_id: [0 1 2 3 4]
Уникальные значения столбца debt: [0 1]


# Вывод

Были выделены основные столбцы, значения в которых важны для ответа на поставленые поромы, значения в столбце total_income заменили на медиану по столбцу, т. к. это столбец важен для ответов на поставленные вопросы

Пропуски могли появиться при "объединении" данных из нескольких источников. Явной закономерности появления пропусков я не обнаружил.

В столбце gender было обнаружено значение 'XNA', с ним ничегоне не делаем, т. к. этот столец не влияет на результаты исследования, а остальные данные в строке с этим значением корректы.

### Замена типа данных

In [200]:
# проанализуруем столбец children на уникальные значения
print(bank_data['children'].unique())
# значение "-1" заменим на "1", т. к. минус одного ребёнка не может быть
bank_data['children'] = bank_data['children'].replace(-1, 1)
print(bank_data[bank_data['children'] == -1]['children'].count())

# столбец total_income и days_employed имеют тип float64, заменим на целочесленный тип, что бы получить данные в рублях / днях
bank_data['total_income'] = bank_data['total_income'].astype('int')
bank_data['days_employed'] = bank_data['days_employed'].astype('int')

[ 1  0  3  2  4 20  5]
0


In [201]:
# как видим в столбце children, есть значения "20", примем их за выброс
# посчитаем колличество таких значений
print(bank_data[bank_data['children'] == 20]['children'].count())
# получаем 76 значений равных "20"
print('Процент людей с 20 детьми от общего количества {:.2%}'.format((bank_data[bank_data['children'] == 20]['children'].count())/len(bank_data)))
# таких значений 0,35%, соответственно их можно удалить

bank_data = bank_data.drop(bank_data[bank_data['children'] == 20].index).reset_index(drop = True)

76
Процент людей с 20 детьми от общего количества 0.35%


### Вывод

Отрицательные значения в столбце children заменяем на положительные, значений не много, поэтому не повлияют сильно на выводы

Значения в столбце total_income с вещественного типа изменяем на целочисленный, что бы получить значения в рублях

### Обработка дубликатов

In [202]:
# приведём данные к одному регистру
bank_data['education'] = bank_data['education'].str.lower()
bank_data['family_status'] = bank_data['family_status'].str.lower()

# посчитаем количество дубликатов
print('Количество дубликатов:', bank_data.duplicated().sum())

# мы нашли 71 дубликат, удалим их
bank_data = bank_data.drop_duplicates().reset_index(drop = True)

Количество дубликатов: 71


### Вывод

Нашли и удалили дубликаты в датафрейме (71 дубликат)

Дубликаты могли появиться при "объединении" данных из нескольких источников. Явной закономерности появления дубликатов я не обнаружил.

### Лемматизация

In [204]:
# получаем список всех категорий
#print(bank_data['purpose'].value_counts())

# выделяем уникальные леммы и считаем частоту их упоминаний

all_lemmas = []

for purpose in bank_data['purpose'].unique():
    all_lemmas += m.lemmatize(purpose)

print(Counter(all_lemmas))

def purpose_category(row):
    string_purpose = row['purpose']
    lemmas = m.lemmatize(string_purpose)
    
    def final_category(lemma):
        if ('свадьба' in lemmas):
            return 'Займ на свадьбу'
        if ('жилье' in lemmas) or ('недвижимость' in lemmas):
            return 'Займ на покупку недвижимости'
        if 'образование' in lemmas:
            return 'Займ на образование'
        if 'автомобиль' in lemmas:
            return 'Займ на покупку автомобиля'
    return final_category(lemmas)


bank_data['category'] = bank_data.apply(purpose_category, axis = 1)

print('Разделяем все цели на 4 категории:')
print(bank_data['category'].value_counts())

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})
Разделяем все цели на 4 категории:
Займ на покупку недвижимости    10775
Займ на покупку автомобиля       4290
Займ на образование              3998
Займ на свадьбу                  2315
Name: category, dtype: int64


### Вывод

Применяем функцимю лемматизации к столбцу purpose, получаем столбец category - с обобщёнными категориями (всего выделено 4 категории)

### Категоризация данных

In [205]:
question_one = bank_data.pivot_table(index = 'children', columns = 'debt', values = 'dob_years', aggfunc = 'count')
question_one.columns = ['Без долга', 'С долгом']
question_one['Доля должников'] = (question_one['С долгом']/(question_one['С долгом'] + question_one['Без долга'])).map('{:.2%}'.format)

question_two = bank_data.pivot_table(index = 'family_status', columns = 'debt', values = 'dob_years', aggfunc = 'count')
question_two.columns = ['Без долга', 'С долгом']
question_two['Доля должников'] = (question_two['С долгом']/(question_two['С долгом'] + question_two['Без долга'])).map('{:.2%}'.format)

question_three_temp = bank_data[['total_income', 'debt', 'dob_years']].copy()
percentile = bank_data['total_income'].describe()

def total_income_and_debt(row):
    total_income = row['total_income']
    debt = row['debt']
    if (total_income <= percentile[4]):
        return '< 107 528'
    if (total_income > percentile[4]) and (total_income <= percentile[5]):
        return '107 528 - 142 594'
    if (total_income > percentile[5]) and (total_income <= percentile[6]):
        return '142 594 - 195 795'
    if (total_income > percentile[6]):
        return '> 195 795'

question_three_temp['total_income_category'] = question_three_temp.apply(total_income_and_debt, axis = 1)

question_three = question_three_temp.pivot_table(index = 'total_income_category', columns = 'debt', values = 'dob_years', aggfunc = 'count')
question_three.columns = ['Без долга', 'С долгом']
question_three['Доля должников'] = (question_three['С долгом']/(question_three['С долгом'] + question_three['Без долга'])).map('{:.2%}'.format)

question_four = bank_data.pivot_table(index = 'category', columns = 'debt', values = 'dob_years', aggfunc = 'count')
question_four.columns = ['Без долга', 'С долгом']
question_four['Доля должников'] = (question_four['С долгом']/(question_four['С долгом'] + question_four['Без долга'])).map('{:.2%}'.format)

### Вывод

Провели категоризацию данных, написали функции для рассчёта значений, необходимых для ответов на поставленные вопросы

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [206]:
print(question_one)

          Без долга  С долгом Доля должников
children                                    
0           13028.0    1063.0          7.54%
1            4410.0     445.0          9.17%
2            1858.0     194.0          9.45%
3             303.0      27.0          8.18%
4              37.0       4.0          9.76%
5               9.0       NaN           nan%


### Вывод

Исходя из полученных данных делаем вывод что люди без детей возвращают кредиты лучше, чем те, у кого есть дети. Между количеством детей и долей должников прямой зависимости нет, также можно выделить "интересный" момент - у кого 5-ть детей - нет долгов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [207]:
print(question_two)

                       Без долга  С долгом Доля должников
family_status                                            
в разводе                   1109        84          7.04%
вдовец / вдова               892        63          6.60%
гражданский брак            3754       385          9.30%
женат / замужем            11362       928          7.55%
не женат / не замужем       2528       273          9.75%


### Вывод

Из полученных данных можем сделать следующие выводы: наиболее благополучними клиентами являются вдовы / вдовцы; наименее благополучными клиентами являются люди в гражданском браке и не женатые / не замужные.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [208]:
print(question_three)

                       Без долга  С долгом Доля должников
total_income_category                                    
107 528 - 142 594           4981       480          8.79%
142 594 - 195 795           4782       445          8.51%
< 107 528                   4918       427          7.99%
> 195 795                   4964       381          7.13%


### Вывод

Исходя из полученных данных делаем вывод что наиболее "благополучная" группа людей с доходом более 195 795 рублей.

Наиболее "неблагополучными" группами являются люди с доходом 142 594 - 195 795 рубле и люди с доходом 107 528 - 142 594 рублей.

- Как разные цели кредита влияют на его возврат в срок?

In [209]:
print(question_four)

                              Без долга  С долгом Доля должников
category                                                        
Займ на образование                3629       369          9.23%
Займ на покупку автомобиля         3889       401          9.35%
Займ на покупку недвижимости       9995       780          7.24%
Займ на свадьбу                    2132       183          7.90%


### Вывод

Исходя из полученных значений можно сделать вывод что наиболее "благополучными" в плане возвратов являются кредиты на недвижисмость. Наименее "благополучными" являются кредины на образование и автомобиль.

### Шаг 4. Общий вывод

Исходя из полученных данных составим портрет наиболее и наименее "благополучного заёмщика"

Наиболее благополучный: человек без детей, вдова / вдовец, с доходом более 195.795 рублей, берущий кредит на покупку недвижисоти.

Наименее благополучный: люди, не женатые / не замужные, с одним или несколькими детьми, с доходом 107.528 - 195.795 рублей, цель кредита - образование или покупка автомобиля. 